# Milestone 1 – User Authentication System  
## PolicyNav – Public Policy Navigation Using AI  

This notebook documents the implementation of a **secure user authentication system**
using:

- Streamlit (Frontend UI)
- MySQL (Database)
- JWT (Authentication)
- Ngrok (Public URL)

This is part of **Infosys Springboard Virtual Internship – Milestone 1**.


In [5]:
!pip install streamlit mysql-connector-python PyJWT pyngrok --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.8 MB/s eta 0:00:00


In [ ]:
# Run this SQL in your local MySQL (not in Colab)

"""
CREATE DATABASE policynav;

USE policynav;

CREATE TABLE users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(100) NOT NULL,
    email VARCHAR(150) UNIQUE NOT NULL,
    password VARCHAR(255) NOT NULL,
    security_question VARCHAR(255) NOT NULL,
    security_answer VARCHAR(255) NOT NULL
);
"""


In [6]:
db_code = """
import mysql.connector

def get_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="YOUR_MYSQL_PASSWORD",
        database="policynav"
    )
"""

with open("db.py", "w") as f:
    f.write(db_code)

print("db.py created")


db.py created


In [7]:
auth_code = """
import jwt
import datetime

SECRET_KEY = "policynav_secret_key"

def create_token(email):
    payload = {
        "email": email,
        "exp": datetime.datetime.utcnow() + datetime.timedelta(hours=2)
    }
    return jwt.encode(payload, SECRET_KEY, algorithm="HS256")
"""

with open("auth.py", "w") as f:
    f.write(auth_code)

print("auth.py created")


auth.py created


In [8]:
app_code = """
import streamlit as st
import re
from db import get_connection
from auth import create_token

st.set_page_config(page_title="PolicyNav Authentication", layout="centered")

if "logged_in" not in st.session_state:
    st.session_state.logged_in = False
if "username" not in st.session_state:
    st.session_state.username = ""
if "email" not in st.session_state:
    st.session_state.email = ""

menu = ["Signup", "Login", "Forgot Password"]
choice = st.sidebar.selectbox("Menu", menu)

# SIGNUP
if choice == "Signup":
    st.title("User Signup")

    username = st.text_input("Username")
    email = st.text_input("Email")
    password = st.text_input("Password", type="password")
    confirm = st.text_input("Confirm Password", type="password")

    question = st.selectbox(
        "Security Question",
        [
            "What is your pet name?",
            "What is your mother’s maiden name?",
            "What is your favorite teacher?"
        ]
    )

    answer = st.text_input("Security Answer")

    if st.button("Signup"):

        username = username.strip()
        email = email.strip()
        password = password.strip()
        confirm = confirm.strip()
        answer = answer.strip()

        if username == "" or email == "" or password == "" or confirm == "" or answer == "":
            st.error("All fields are mandatory")

        elif not re.match(r"[^@]+@[^@]+\\.[a-zA-Z]{2,}", email):
            st.error("Invalid email format")

        elif not password.isalnum():
            st.error("Password must be alphanumeric")

        elif password != confirm:
            st.error("Passwords do not match")

        else:
            conn = get_connection()
            cursor = conn.cursor()

            try:
                cursor.execute(
                    "INSERT INTO users (username,email,password,security_question,security_answer) VALUES (%s,%s,%s,%s,%s)",
                    (username, email, password, question, answer)
                )
                conn.commit()

                token = create_token(email)

                st.success("Signup successful")
                st.code(token)

            except:
                st.error("Email already exists")

            finally:
                conn.close()

# LOGIN
elif choice == "Login":
    st.title("User Login")

    if not st.session_state.logged_in:

        email = st.text_input("Email")
        password = st.text_input("Password", type="password")

        if st.button("Login"):
            conn = get_connection()
            cursor = conn.cursor()

            cursor.execute("SELECT username, password FROM users WHERE email=%s", (email,))
            user = cursor.fetchone()
            conn.close()

            if user and user[1] == password:
                st.session_state.logged_in = True
                st.session_state.username = user[0]
                st.session_state.email = email
                st.session_state.token = create_token(email)
                st.success("Login successful")
                st.rerun()
            else:
                st.error("Invalid credentials")

# DASHBOARD
if st.session_state.logged_in:
    st.title("Dashboard")
    st.subheader(f"Welcome, {st.session_state.username}")
    st.code(st.session_state.token)

    if st.button("Logout"):
        st.session_state.clear()
        st.rerun()

# FORGOT PASSWORD
if choice == "Forgot Password" and not st.session_state.logged_in:
    st.title("Forgot Password")
"""
with open("app.py", "w") as f:
    f.write(app_code)

print("app.py created")


app.py created


## Running the Application Locally

1. Download this project to your laptop  
2. Ensure MySQL is running with the **policynav** database  
3. Run:




4. Start ngrok:ngrok http 8501



5. Use the **HTTPS public URL** for demonstration.
